# Q-Learning
## Elia Fantini (336006) & Félix Klein (344259)

In [28]:
MAX_STICKS=7
NUM_HEAPS=3
METRICS_FREQUENCY=250

In [29]:
# basic libraries
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

# data visualization and parallel computation
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import multiprocessing as mp
from joblib import Parallel, delayed

# recovering results so everything does not have to be re-run
import pickle
use_backup = False
if use_backup:
    file_name = "q_learning_results.pkl"
    open_file = open(file_name, "rb")
    training_results = pickle.load(open_file)
    open_file.close()
else:
    training_results = []

from nim_env import NimEnv, OptimalPlayer


def heaps_to_idx(heaps):
    """
    Converts heaps state in the corresponding index of the row in the Q-table
    Args:
        heaps: list of integers
            list of heap sizes.

    Returns:
        idx: int
            index of the state's row in the Q-table
    """
    idx=0
    for i in range(NUM_HEAPS):
        idx += heaps[i]*((MAX_STICKS + 1)**(NUM_HEAPS-i-1))
    return idx

def idx_to_heaps(idx):
    """
    Converts index of the row in Q-table to the corresponding heaps' state
    Args:
        idx: int
            index of the state's row in the Q-table

    Returns:

        heaps: list of integers
            list of heap sizes.
    """
    idx_copy=idx
    heaps=[]
    for i in range(NUM_HEAPS):
        heaps.append(int(idx_copy//((MAX_STICKS + 1)**(NUM_HEAPS-i-1))))
        idx_copy = idx_copy - heaps[i]*((MAX_STICKS + 1)**(NUM_HEAPS-i-1))
    return heaps

def create_q_table(num_heaps, max_sticks, initial_value=0.0):
    """
    Creates a list of numpy.ndarray, one for every possible state. Each ndarray is initialized with initial value.
    Args:
        num_heaps: int
            number of heaps
        max_sticks: int
            number of max sticks possible in a heap
        initial_value: float
            initial q-values value. Default is 0.0

    Returns:

        q-table: list of numpy.ndarray
            matrix of q-values, initialized with initial_value
    """
    q_table=[]
    for i in range((max_sticks+1)**num_heaps):
        if i==0:
            q_table.append(np.full(shape=[1,1], fill_value=initial_value,dtype='float64'))
            continue
        else:
            heaps = idx_to_heaps(i)
            num_actions=0
            for ii in range(num_heaps):
                num_actions += heaps[ii]
            q_table.append(np.full(shape=[num_actions,1], fill_value=initial_value,dtype='float64'))
    return q_table

def play_game(env,q_table, player_q, player_adv, eta, gamma, test= False, self_learning= False):
    """
    Plays a full nim match, till one of the two players loses.
    Args:
        env: NimEnv
            instance of the NimEnv class to play the game with
        q_table: list on numpy.ndarray
            matrix containing q-values to apply q-learning algorithm
        player_q: OptimalPlayer
            q-learning agent
        player_adv: OptimalPlayer
            optimal policy agent
        eta: float
            number that controls how much the q-values should vary during training
        gamma: float
            number that controls how much next expected rewards should count on the learning update
        test: bool
            if True, the learning process will be tested by playing against the optimal player without updating q-values. Default is False
        self_learning: bool
            if True, player_adv will be considered a q-learning agent as well, so the learning agent will play against itself. Default is False

    Returns:
        reward: int
            1 if player-q won, -1 otherwise

    """
    # resetting the variables and the NimEnv
    seed = random.uniform(0,9999)
    env.reset(seed)
    heaps, _, __ = env.observe()
    reward = 0 # stays at 0 until game is finished
    while not env.end:
        if env.current_player == player_q.player:   # player 1 : learning agent
            # q-learning agent 1 choosing next action using q-values : making the move
            state_idx = heaps_to_idx(heaps)
            move, q_value, action = player_q.act_q(heaps, q_table[state_idx], greedy= False) # if greedy = False, it means eps-greedy
            heaps, end, winner = env.step(move)
            if self_learning and not test:
                # q-learning agent 2 choosing next potential greedy-action using q-values, without actually taking it, then updating the q-table
                next_state_ind_adv = heaps_to_idx(heaps)
                _,q_value_next_adv,_= player_adv.act_q(heaps, q_table[next_state_ind_adv], greedy= True)
                if not end and env.num_step>1:
                    q_table[state_idx_adv][action_adv] += eta * (reward + gamma * q_value_next_adv - q_value_adv)


        else:                                       # optimal player OR other learning agent if not self_learning
            if not self_learning or test:
                # choosing next action using the optimal policy
                move = player_adv.act(heaps)
                heaps, end, winner = env.step(move)
            if not test:
                if self_learning:
                    # q-learning agent 2 choosing next action using q-values
                    state_idx_adv = heaps_to_idx(heaps)
                    move,q_value_adv,action_adv= player_adv.act_q(heaps, q_table[state_idx_adv], greedy= False)
                    heaps, end, winner = env.step(move)
                # q-learning agent 1 choosing next potential greedy-action using q-values, without actually taking it, then updating the q-table
                next_state_ind = heaps_to_idx(heaps)
                _,q_value_next,_= player_q.act_q(heaps, q_table[next_state_ind], greedy= True)
                if not end and env.num_step>1:
                    q_table[state_idx][action] += eta * (reward + gamma * q_value_next - q_value)


        if end:
            if winner== player_q.player:
                reward = 1
            else:
                reward = -1
            if not test:
                # q-learning agent 1 updating q-table with final reward value
                q_table[state_idx][action] += eta * (reward + gamma * q_table[0][0] - q_value)
                if self_learning:
                    # q-learning agent 2 updating q-table with final reward value
                    q_table[state_idx_adv][action_adv] += eta * (-reward + gamma * q_table[0][0] - q_value_adv)
            break
    return reward


def q_learning(env = None, q_table=None, num_games= 20000, eta=0.1, gamma=0.99, eps_q=0.2, eps_adv= 0.5, initial_table_value=0, decreasing_eps= False, eps_min=0.1, eps_max=0.8, n_star=1, test=False, verbose= False, self_learning=False):
    """
    Main function that handles the whole training. It runs num_games Nim matches and updates the
    q-table according to q-learning algorithm. It also calculates statistics of the learning
    process, such as the mean of the learning agent's rewards, computed every METRICS_FREQUENCY games.
    If test is True, also M-opt and M-rand metrics are computed. It returns a list of the metrics and the
    trained q-table.
    Args:

        env: NimEnv
            instance of the NimEnv class to play the game with. Default is None
        q_table: list on numpy.ndarray
            matrix containing q-values to apply q-learning algorithm. Default is None
        num_games: int
            num of games to be played to train the learning agent.Default is 20000
        eta: float
            number that controls how much the q-values should vary during training. Default is 0.1
        gamma: float
            number that controls how much next expected rewards should count on the learning update. Default is 0.99
        eps_q: float
            probability for the q-learning agent to perform a random action instead of the greedy one. Default is 0.2
        eps_adv: float
            probability for the adversarial agent to perform a random action instead of the optimal one. Default is 0.5
        initial_value: float
            initial q-values value. Default is 0.0
        decreasing_eps: bool
            if True, the learning agent eps value will decrease over the number of games played n. eps-q will be ignored.
            eps(n) = max(eps_min, eps_max*(1 −n/n_star). . Default is False
        eps_min: float
            minimum eps value that decreasing eps can assume. Default is 0.1
        eps_max: float
             maximum eps value that decreasing eps can assume. Default is 0.8
        n_star: int
            number that controls how slow the decreasing eps reaches its minimum. Default is 1
        test: bool
            if True, the learning process will be tested every METRICS_FREQUENCY training games by playing 500 games
            with frozen q-values against Opt(0), the optimal player, and 500 against Opt(1), the random player. Default is False
        verbose: bool
            if  True, it prints a progress bar. Default is False
        self_learning: bool
            if True, player_adv will be considered a q-learning agent as well, so the learning agent will play against itself. Default is False

    Returns:
        metrics: list of lists of float
            list containing a list of mean reward values. If test is True, it also contains a list of M-opt values and a list of M-rand values.
            Such values are computed and appended to the list every METRICS_FREQUENCY games
        q_table: list of numpy.ndarray
            the q-table with the learned q-values

    """

    if env is None:
        env = NimEnv()
    if q_table is None:
        q_table = create_q_table(NUM_HEAPS,MAX_STICKS,initial_table_value)

    # initializing metrics
    rewards = [] # -1 or 1, stored at the end of each game
    rewards_mean=[] # mean of every x rewards (here 250) stored after the x_th game
    m_opt=[]
    m_rand=[]

    Turn = 0
    # both the agent and the adversary are OptimalPlayer instances but the agent is calling act_q whereas adv is calling act
    # if self_learning = True, both are calling act_q
    for i in tqdm(range(num_games), disable=(not verbose)):
        if decreasing_eps:
            eps = max(eps_min,eps_max*(1-i/n_star))
            player_q = OptimalPlayer(epsilon=eps, player=Turn)
        else:
            eps = eps_q
            player_q = OptimalPlayer(epsilon=eps, player=Turn)
        if self_learning:
            # at the beginning of every game, reinstantiate the optimal player (especially if eps changes, turn changes anyway)
            player_adv = OptimalPlayer(epsilon=eps, player= 1-Turn)
        else:
            player_adv = OptimalPlayer(epsilon=eps_adv, player=1-Turn)
        rewards.append(play_game(env, q_table, player_q, player_adv, eta, gamma, False, self_learning=self_learning))
        if i%METRICS_FREQUENCY==0 and i!=0:
            rewards_mean.append(np.asarray(rewards).mean())
            rewards = []
            if test:    # plays against optimal player (OPT(0)) and random player (OPT(1))
                for eps_opt in [0,1]:
                    won_count=0
                    for ii in range(500):
                        heaps, _, __ = env.observe()
                        player_q = OptimalPlayer(epsilon=0, player=Turn)
                        player_opt = OptimalPlayer(epsilon=eps_opt, player=1-Turn)
                        if 1 == play_game(env, q_table, player_q, player_opt, eta, gamma, True):
                            won_count +=1
                        Turn = 1 - Turn
                    m=(won_count-(500-won_count))/500
                    if eps_opt==0:
                        m_opt.append(m)
                    else:
                        m_rand.append(m)
        Turn = 1 - Turn
    metrics = []
    metrics.append(rewards_mean)
    if test:
        # list of three lists, agent, optimal player and random player
        metrics.append(m_opt)
        metrics.append(m_rand)
    return metrics, q_table

## 2.1 Learning from experts
In this section, you will study whether Q-learning can learn to play Nim by playing against Opt(opt)
for some opt ∈ [0, 1]. To do so, implement the Q-learning algorithm. To check the algorithm, run a
Q-learning agent, with a fixed and arbitrary eps ∈ [0, 1), against Opt(0.5) for 20’000 games – switch the
1st player after every game. Note that the agents are not allowed to take an unavailable action, i.e. they
cannot take more than existing number of sticks in a heap from that heap.
Question 1. Plot average reward for every 250 games during training – i.e. after the 250th game, plot
the average reward of the first 250 games, after the 500th game, plot the average reward of games 251 to
500, etc. Does the agent learn to play Nim?

In [30]:
env = NimEnv()
metrics,_=q_learning(env,num_games=20000, eps_q=0.1, gamma= 0.99, verbose=False)
x = np.arange(1,len(metrics[0])+1)*METRICS_FREQUENCY
fig = px.line(x=x, y=metrics, title=f'Average reward over time of RL agent with policy epsilon={0.1}')
fig.update_layout(xaxis_title='Games played', yaxis_title='Average reward')
fig.update_layout(showlegend=False)
fig.update_layout(width=600)
fig.show()

training_results.append([x,metrics])

The learning agent plays against a player that does the best possible move half of the times, whereas the other half of the times it plays a random move. Given that every game the first player is switched, that heaps are randomly generated with a uniform distribution and mathematically is already possible to know who will win among two Opt(0) players just by knowing initial heaps and starting player, the probability of winning for each player is around 0.5 and hence the mean reward would be around 0.0, half of the game won (+1) and half lost (-1). Given that the adversary is Opt(0.5), even when Opt(0) is supposed to lose has a 50% chance that adversary will do a wrong move, so the expect win percentage is 75% and the mean reward is 0.5. Since the learning agent reaches such mean reward, it seems like it really has learned to play with the optimal policy, getting the same mean reward a Opt(0) agent would have reached.

## 2.1.1 Decreasing exploration
One way to make training more efficient is to decrease the exploration level eps over time. If we define eps(n)
to be eps for game number n, then one feasible way to decrease exploration during training is to use
eps(n) = max{eps_min, eps_max(1 −n/n∗)},where eps_min and eps_max are the minimum and maximum values for eps, respectively, and n∗ is the number of
exploratory games and shows how fast eps decreases. For convenience, we assume eps_min = 0.1 and eps_max = 0.8;Use eps(n) as define above and
run different Q-learning agents with different values of n∗ against Opt(0.5) for 20’000 games – switchthe 1st player after every game. Choose several values of n∗ from a reasonably wide interval between 1 to 40’000 – particularly, include n
∗ = 1.
Question 2. Plot average reward for every 250 games during training. Does decreasing eps help training
compared to having a fixed eps? What is the effect of n∗?

In [31]:
env = NimEnv()
#output=[]
n_star_values=np.linspace(1,40000,6)

df = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
num_cores = min(len(n_star_values),mp.cpu_count())
def parallelGames(n_star):
    df = {}
    metrics,_=q_learning(env,num_games=20000, eps_q=0.1, gamma=0.99, decreasing_eps=True, n_star=n_star, verbose=False)
    #y=metrics.reshape(metrics.size//METRICS_FREQUENCY, METRICS_FREQUENCY).mean(axis=1)
    df[f'n*={n_star:.0f}'] = metrics[0]
    return df

dfs = Parallel(n_jobs=num_cores)(delayed(parallelGames)(n_star) for n_star in n_star_values)
for d in dfs:
    df.update(d)

df = pd.DataFrame(df)
fig = px.line(df, x='Games played', y=df.columns.difference(['Games played']), title=f'Training with Decreasing epsilon. Comparing different n* values')
fig.update_layout(width=800)
fig.update_yaxes(title='Mean reward')
fig.show()

training_results.append([df['Games played'],df.columns.difference(['Games played'])])

It seems like n*=1 gives an extra boost in learning in the beginning, making the mean reward curve grow faster. Higher values seem to have similar results to fixed eps and n*>6000 means a slower learning curve but explores more possibilities. The sweet spot is converging to the same mean reward in 20'000 games while achieving a lower generalization error.

Question 3. After every 250 games during training, compute the ‘test’ Mopt and Mrand for your agents
– when measuring the ‘test’ performance, put eps = 0 and do not update the Q-values. Plot Mopt and
Mrand over time. Describe the differences and the similarities between these curves and the ones of the
previous question.

M_opt/rand = (N_win - N_win)/ N_tot

In [32]:
env = NimEnv()
metrics,_=q_learning(env,num_games=20000, eps_q=0.1, gamma= 0.99, test=True, verbose=False)
x = np.arange(1,len(metrics[0])+1)*METRICS_FREQUENCY
df = {'Games played':list(x),
    'Mean reward':metrics[0],
    'M_opt':metrics[1],
    'M_rand':metrics[2]}
df = pd.DataFrame(df)
fig = px.line(df,x='Games played', y=df.columns.difference(['Games played']), title=f'Training with testing')
fig.update_layout(xaxis_title='Games played', yaxis_title='Reward')
fig.update_layout(width=600)
fig.show()

training_results.append([x,df.columns.difference(['Games played'])])

The learning agent has certainly learned something, since it always manages to beat a random player. On the other hand it loses around 3 out of 4 times against Opt(0), maybe because it learned from a Opt(0.5) player that half of the times did a random and likely wrong move

## 2.1.2 Good experts and bad experts
Choose the best value of n∗ that you found in the previous section. Run Q-learning against Opt(eps_opt) for
different values of eps_opt for 20’000 games – switch the 1st player after every game. Choose several values
of eps_opt from a reasonably wide interval between 0 to 1 – particularly, include eps_opt = 0.

Question 4. After every 250 games during training, compute the ‘test’ Mopt and Mrand for your agents
– for each value of eps_opt. Plot Mopt and Mrand over time. What do you observe? How can you explain it?


In [33]:
env = NimEnv()
#output = []
eps_adv_values = np.linspace(0,1,11)
# redefining the parallelGames function for M_opt and M_rand configuration
df1 = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
df2 = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
df3 = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
num_cores = mp.cpu_count()
def parallelGames(eps):
    df1, df2, df3 = {}, {}, {}
    metrics, _=q_learning(env,num_games=20000, eps_adv=eps, gamma= 0.99, decreasing_eps=True, n_star=1, verbose=False, test=True)
    df1[f"eps={eps:0.2f}"] = metrics[0]
    df2[f"eps={eps:0.2f}"] = metrics[1]
    df3[f"eps={eps:0.2f}"] = metrics[2]
    return df1, df2, df3
dfs = Parallel(n_jobs=num_cores)(delayed(parallelGames)(eps_adv) for eps_adv in eps_adv_values)
for d1,d2,d3 in dfs:
    df1.update(d1)
    df2.update(d2)
    df3.update(d3)

df1, df2, df3 = pd.DataFrame(df1), pd.DataFrame(df2), pd.DataFrame(df3)
fig1 = px.line(df1, x='Games played', y=df1.columns.difference(['Games played']), title="Training with testing on different adversary's eps values")
fig1.update_layout(width=800)
fig1.update_yaxes(title_text = "Mean reward")
fig2 = px.line(df2, x='Games played', y=df2.columns.difference(['Games played']))
fig2.update_layout(width=800)
fig2.update_yaxes(title_text = "Mopt")
fig3 = px.line(df3, x='Games played', y=df3.columns.difference(['Games played']))
fig3.update_layout(width=800)
fig3.update_yaxes(title_text = "Mrand")
fig1.show()
fig2.show()
fig3.show()

training_results.append([df1['Games played'], df1.columns.difference(['Games played'])])
training_results.append([df2['Games played'], df2.columns.difference(['Games played'])])
training_results.append([df3['Games played'], df3.columns.difference(['Games played'])])

Has seen before, a high eps value for Opt(eps) adversary gives poor learning performances when tested against Opt(0), so a lower eps is needed. 0.1, 0.05 and 0.0 seem the best, but eps=0.0 performs poorly against the random player. I CAN'T REALLY EXPLAIN WHY

## 2.2 Learning by self-practice
In this section, your are supposed to ask whether Q-learning can learn to play Nim by only playing
against itself. For different values of eps ∈ [0, 1), run a Q-learning agent against itself for 20’000 games –
i.e. both players use the same set of Q-values and update the same set of Q-values.

Question 7. After every 250 games during training, compute the ‘test’ Mopt and Mrand for different
values of eps ∈ [0, 1). Does the agent learn to play Nim? What is the effect of eps?

In [34]:
env = NimEnv()
eps_values=np.linspace(0, 1, 11)
# redefining the parallelGames function for M_opt and M_rand configuration
df1 = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
df2 = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
df3 = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
num_cores = mp.cpu_count()
def parallelGames(eps):
    df1, df2, df3 = {}, {}, {}
    metrics, _ = q_learning(env, num_games=20000, eps_q=eps, gamma=0.99, verbose=False, test=True, self_learning=True)
    df1[f"eps={eps:0.2f}"] = metrics[0]
    df2[f"eps={eps:0.2f}"] = metrics[1]
    df3[f"eps={eps:0.2f}"] = metrics[2]
    return df1, df2, df3
dfs = Parallel(n_jobs=num_cores)(delayed(parallelGames)(eps_adv) for eps_adv in eps_adv_values)
for d1,d2,d3 in dfs:
    df1.update(d1)
    df2.update(d2)
    df3.update(d3)

df1, df2, df3 = pd.DataFrame(df1), pd.DataFrame(df2), pd.DataFrame(df3)
fig1 = px.line(df1, x='Games played', y=df1.columns.difference(['Games played']), title="Training and testing with self-learning, using different eps values")
fig1.update_layout(width=800)
fig1.update_yaxes(title_text = "Mean reward")
fig2 = px.line(df2, x='Games played', y=df2.columns.difference(['Games played']))
fig2.update_layout(width=800)
fig2.update_yaxes(title_text = "Mopt")
fig3 = px.line(df3, x='Games played', y=df3.columns.difference(['Games played']))
fig3.update_layout(width=800)
fig3.update_yaxes(title_text = "Mrand")
fig1.show()
fig2.show()
fig3.show()

training_results.append([df1['Games played'], df1.columns.difference(['Games played'])])
training_results.append([df2['Games played'], df2.columns.difference(['Games played'])])
training_results.append([df3['Games played'], df3.columns.difference(['Games played'])])

Since the players during training are the same, it makes sense that they have the same probability of winning, hence the mean reward always oscillates around 0.0. Again, the learner always learns something since it's always better than the random player, but only low eps values seem to learn and play as a Opt(0) player. Best eps values are the same as before when learning against a Opt(eps) player and not against itself, but the low M-rand with eps= 0.0 is not there anymore!

Question 8. After every 250 games during training, compute the ‘test’ Mopt and Mrand for your agents.
Does decreasing eps help training compared to having a fixed eps? What is the effect of n∗?

In [35]:
env = NimEnv()
output = []
n_star_values=np.linspace(1, 40000, 6)

# redefining the parallelGames function for M_opt and M_rand configuration
df1 = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
df2 = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
df3 = {'Games played':list(np.arange(1,20000//METRICS_FREQUENCY)*METRICS_FREQUENCY)}
num_cores = mp.cpu_count()
def parallelGames(n_star):
    df1, df2, df3 = {}, {}, {}
    metrics,_ = q_learning(env, num_games=20000, gamma=0.99, decreasing_eps=True,
                                        n_star=n_star, verbose=False, test=True, self_learning=True)
    df1[f"Average reward (n*={n_star:.0f})"] = metrics[0]
    df2[f"Mopt (n*={n_star:.0f})"] = metrics[1]
    df3[f"Mrand (n*={n_star:.0f})"] = metrics[2]
    return df1, df2, df3
dfs = Parallel(n_jobs=num_cores)(delayed(parallelGames)(n_star) for n_star in n_star_values)
for d1,d2,d3 in dfs:
    df1.update(d1)
    df2.update(d2)
    df3.update(d3)

df1, df2, df3 = pd.DataFrame(df1), pd.DataFrame(df2), pd.DataFrame(df3)
fig1 = px.line(df1, x='Games played', y=df1.columns.difference(['Games played']), title="Training and testing with self-learning and decreasing eps, using different n* values")
fig1.update_layout(width=800)
fig1.update_yaxes(title_text = "Mean reward")
fig2 = px.line(df2, x='Games played', y=df2.columns.difference(['Games played']))
fig2.update_layout(width=800)
fig2.update_yaxes(title_text = "Mopt")
fig3 = px.line(df3, x='Games played', y=df3.columns.difference(['Games played']))
fig3.update_layout(width=800)
fig3.update_yaxes(title_text = "Mrand")
fig1.show()
fig2.show()
fig3.show()

training_results.append([df1['Games played'], df1.columns.difference(['Games played'])])
training_results.append([df2['Games played'], df2.columns.difference(['Games played'])])
training_results.append([df3['Games played'], df3.columns.difference(['Games played'])])

It doesn't seem to help

Question 9. What are the highest values of Mopt and Mrand that you could achieve after playing 20’000
games?
Question 10. For three board arrangements (i.e. states s), visualize Q-values of available actions (e.g.
using heat maps). Does the result make sense? Did the agent learn the game well?

In [36]:
env = NimEnv()
metrics,q_values = q_learning(env, num_games=20000, gamma=0.99, decreasing_eps=True,
                                    n_star=1, verbose=False, test=True, self_learning=True)
x = np.arange(1,len(metrics[0])+1)*METRICS_FREQUENCY
df = {'Games played':list(x),
    'Mean reward':metrics[0],
    'M_opt':metrics[1],
    'M_rand':metrics[2]}
df = pd.DataFrame(df)
fig = px.line(df,x='Games played', y=df.columns.difference(['Games played']), title=f'Training with self-learning')
fig.update_layout(xaxis_title='Games played', yaxis_title='Average reward')
fig.update_layout(width=600)
fig.show()

training_results.append([x, df.columns.difference(['Games played'])])

In [37]:
heaps=[[4,0,0],[1,2,0],[1,0,1]]
for i in range(3):
    state = heaps_to_idx(heaps[i])
    fig = px.imshow(np.array(q_values[state]), color_continuous_scale='gray', title=str(heaps[i]))
    fig.update_layout(height=400, width=400)
    fig.show()

The values make sense: in the first case the move that makes you win instantly has a 0.9 q-value. In the second case two moves makes you lose in the very next move and one doesn't, and such move has a 0.9 q-value. Last case, both moves make you lose and they both have a -1 q-value.Nice!

In [38]:
file_name = "q_learning_results.pkl"
open_file = open(file_name, "wb")
pickle.dump(training_results, open_file)
open_file.close()